<a href="https://colab.research.google.com/github/purang2/prompting-nlp/blob/main/Fewshot_GPT%26T5_StyleTransfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Manual(수동) Prompt Engineering for Dialog2Prompt with Large Language Model 

## **[ Produced by ]**

[Eunchan Lee of KNU-BrainAI](https://github.com/purang2)




In [1]:
import gc
import torch 
  
gc.collect()
torch.cuda.empty_cache()

In [2]:
!pip install transformers 
!pip install accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 13.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.5/191.5 KB 4.6 MB/s eta 0:00:00


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

Put datapair

In [4]:
few_src = [] 
few_tgt = []


def put_datapair(src, tgt):
  few_src.append(src)
  few_tgt.append(tgt)


# k=4
'''
put_datapair("k", "x")
put_datapair("kk", "xx")
put_datapair("kkk", "xxx")
put_datapair("kkkk", "xxxx")
'''


# k=4 -> Manual sample: Text style transfer
put_datapair("South Korea's Capital is Seoul", "Seoul -> South Korea")
put_datapair("Mexico's Capital is Mexico City", "Mexico City -> Mexico")
put_datapair("Germany's Capital is Berlin", "Berlin -> Germany")
put_datapair("Uganda's Capital is Kampala", "Kampala -> Uganda")



print(few_src)
print(few_tgt)


["South Korea's Capital is Seoul", "Mexico's Capital is Mexico City", "Germany's Capital is Berlin", "Uganda's Capital is Kampala"]
['Seoul -> South Korea', 'Mexico City -> Mexico', 'Berlin -> Germany', 'Kampala -> Uganda']


### Prompt

***Reference: [A Recipe for Arbitrary Text Style Transfer with Large Language Models, (Reif et al. (Google Research), ACL 2022)](https://aclanthology.org/2022.acl-short.94/)***

In [5]:
in_context_text = ''

for i in range(len(few_src)):
  in_context_text += f'Here is some text: {few_src[i]}. Here is a rewrite of the text, which is more simple: {few_tgt[i]}\n'

print(in_context_text)

Here is some text: South Korea's Capital is Seoul. Here is a rewrite of the text, which is more simple: Seoul -> South Korea
Here is some text: Mexico's Capital is Mexico City. Here is a rewrite of the text, which is more simple: Mexico City -> Mexico
Here is some text: Germany's Capital is Berlin. Here is a rewrite of the text, which is more simple: Berlin -> Germany
Here is some text: Uganda's Capital is Kampala. Here is a rewrite of the text, which is more simple: Kampala -> Uganda



transformers model

In [6]:
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from transformers import pipeline


# GPT-2 (This is the smallest version of GPT-2, with 124M parameters.) = "gpt2"
# GPT-2 (XL) (1.5B parameters.) = "gpt2-xl"
# T5 (base, with 220 million parameters.) = "t5-base"
# T5 (3B parameters.) = "t5-3b"
# T5 (11B parameters.) = "t5-11b" -> 45.2GB (Colab may not cover this size..)
# GPT-J (6B parameters.) = "EleutherAI/gpt-j-6B"

# In-context learning work well at least 1 Billions parameters. -eunchan-

model_zoo = ["gpt2","gpt2-xl","t5-base","t5-3b","t5-11b","EleutherAI/gpt-j-6B"]



model_name = model_zoo[1]


if model_name.find('t5') > -1: #model = t5
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
  nlg_pipeline = pipeline('text2text-generation',model=model, tokenizer=tokenizer)

else: #model = gpt
  tokenizer = AutoTokenizer.from_pretrained(model_name)
  model = AutoModelForCausalLM.from_pretrained(model_name)
  #nlg_pipeline = pipeline(model=model, tokenizer=tokenizer)

Downloading:   0%|          | 0.00/689 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

model inference

In [10]:

#test_input_text = 'kjkjkjkj'
test_input_text = "Italy's Capital is Roma"
test_output_length = 8 #token length



if model_name.find('t5') > -1: #model = T5
  def generate_text(pipe, text, num_return_sequences=5, max_length=512):
    text = f"{text}"
    out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length, num_beams=5, no_repeat_ngram_size=2,)
    return [x['generated_text'] for x in out]

  #target_text = 'kjkjkjkj'
  src_text = in_context_text + f"Here is some text: {test_input_text}. Here is a rewrite of the text, which is more simple: "

  print("Input text:", src_text)
  test_output_text = generate_text(nlg_pipeline, src_text, num_return_sequences=1, max_length=test_output_length)


  #you can cook this output anything you want!  
  print(test_output_text)



else:  #model = GPT
  src_text = in_context_text + f"Here is some text: {test_input_text}. Here is a rewrite of the text, which is more simple: "
  tokens = tokenizer.encode(src_text, return_tensors='pt')
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=len(tokens[0])+test_output_length)
  generated = tokenizer.batch_decode(gen_tokens)[0]
  
  test_output_text = generated[generated.rfind('more simple:')+12:]
  print(generated)

  #you can cook this output anything you want!
  print(test_output_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Here is some text: South Korea's Capital is Seoul. Here is a rewrite of the text, which is more simple: Seoul -> South Korea
Here is some text: Mexico's Capital is Mexico City. Here is a rewrite of the text, which is more simple: Mexico City -> Mexico
Here is some text: Germany's Capital is Berlin. Here is a rewrite of the text, which is more simple: Berlin -> Germany
Here is some text: Uganda's Capital is Kampala. Here is a rewrite of the text, which is more simple: Kampala -> Uganda
Here is some text: Italy's Capital is Roma. Here is a rewrite of the text, which is more simple:  Roma -> Italy
Here is
  Roma -> Italy
Here is
